# INF8215 - Intelligence artif.: méthodes et algorithmes 
## Automne 2019 - TP2 - Recherche Adversarielle 
### Membres de l'équipe
    - MÉTAIS Julien
    - SOULIE Achille




## Directives de remise
Le travail sera réalisé avec la  même équipe que pour le TP1. Vous remettrez ce fichier nommé TP2\_NomDuMembre1\_NomDuMembre2\_NomDuMembre3.ipynb dans la boîte de remise sur moodle. 

Tout devra être remis avant le **6 novembre à 23h55**. Tout travail en retard sera pénalisé
d’une valeur de 10\% par jour de retard.

## Barème
Partie 1: 25 points

Partie 2: 35 points

Partie 3: 15 points

Partie 4: 15 points

Partie 5: 10 points

Pour un total de 100 points possibles sur 100 points.


## Recherche Adversarielle

La recherche adversarielle est une méthode d’intelligence artificielle qui est utilisée pour prédire les actions d’un adversaire dans un jeu vidéo. Il consiste à construire un arbre avec les actions possibles suivi des réactions de l’adversaire à ces actions. Pour choisir l’action optimale, il existe plusieurs méthodes. Une très commune est la recherche minimax. Elle évalue la valeur des actions et maximise la valeur de celle du joueur et minimise la valeur de celle de l’adversaire.

### Rush Hour

Vous avez vu le jeu de puzzle, Rush Hour, dans le TP précédent. Dans le cadre de ce TP, un adversaire a été ajouté pour transformer Rush Hour en jeu adversarielle. À chaque tour, l’adversaire ajoute une roche dans la grille. Cette roche disparaît après 1 tour et empêche les voitures de se déplacer. Son but est d’obliger le joueur à faire le plus de coups possible et de l’empêcher de bouger complètement. Le joueur peut déplacer une voiture, selon les règles vues dans le tp précédent, par tour. Cette version de Rush Hour est un jeu zero-sum déterministe.

![Rushhour](https://i.stack.imgur.com/NzztF.jpg)

### But

Votre but dans ce TP est de développer un algorithme minimax pour permettre à votre AI de terminer le niveau de Rush Hour **avec un minimun de mouvement**. Une version complétée des classes Rush Hour et State est fournie pour vous aider à faire le TP. Ces classes vont être réutilisées pour représenter les états à évaluer. Vous pouvez les remplacer par vos propres implémentations en Python (vous pouvez aussi réécrire le TP dans le langage de votre choix).  

## 1. Implémentation de Rush Hour (25pts)
Les classes suivantes sont complètes pour représenter les différents états d’une partie de Rush Hour normale.
Il manque la représentation des coups de l’adversaire : vous auriez à implémenter cette partie-là dans les classes touchées.

L’état du stationnement au début et à tout moment du jeu est représenté par un objet de la classe **State** qui contient les champs suivants : 

- `pos` : un vecteur indiquant la position de chaque véhicule sur la grille (la valeur représente la première case occupée par la voiture);

- `c` : l’index de la voiture déplacée pour atteindre l’état courant à partir de l’état précédent;

- `d` : la direction du déplacement fait à partir de l’état précédent pour atteindre l’état courant (+1 : vers la droite ou vers le bas, -1 : vers la gauche ou vers le haut);

- `prev` : l’état précédent

- `score` : le score associé à l'état

- `rock` : la position de la roche dans la grille (ligne,colonne)

In [43]:
import numpy as np
import math
import copy
import random
from collections import deque
#Comparaison du minimax normal et du minimax avec pruning : 
import statistics as stat

class State:
    
    """
    Contructeur d'un état initial
    """
    def __init__(self, pos):
        """
        pos donne la position de la voiture i dans sa ligne ou colonne (première case occupée par la voiture);
        """
        self.pos = np.array(pos)
        
        """
        c,d et prev premettent de retracer l'état précédent et le dernier mouvement effectué
        """
        self.c = self.d = self.prev = None
        
        self.nb_moves = 0
        self.score = 0
        
        # TODO
        self.rock = None #Position de la roche
        
        self.visitedCars = set() #Pour le calcul du score de l'état : voitures déjà considérées dans la chaîne de blocage
      
    
    """
    Renvoie la liste des voitures bloquant le déplacement de la voiture courante (d'indice idVoiture)
    ainsi que les directions dans lesquelles envoyer ces voitures, le nombre de coups nécessaires pour évacuer chacune d'elles,
    sachant la direction dans laquelle on veut déplacer la voiture courante, et la distance sur laquelle elle doit se déplacer. 
    """ 
    def chercheBloquantes(self, idVoiture, right_down, dist, rh):
        
        estHoriz = rh.horiz[idVoiture]
        bloquantes = [] #Liste des indices des voitures bloquantes
        directionsVoulues = [] #Directions sous la forme de booléens (True <==> right or down)
        minMovesNeeded = [] #Minorants du nombre de déplacements pour évacuer les voitures bloquantes
        
        for i in range(rh.nbcars): #On itère sur les voitures
            
            #La voiture i bloque-t-elle idVoiture?
            estBloquante = False
            minMoves = 0
            
            if (rh.horiz[i] == estHoriz) and (rh.move_on[i] == rh.move_on[idVoiture]): #Cas d'une voiture sur la même ligne/colonne
                if(self.pos[i] > self.pos[idVoiture] and right_down): #La voiture est-elle du côté bloquant?
                    if(self.pos[i] < self.pos[idVoiture] + rh.length[idVoiture] + dist): #Va-t-elle gêner le déplacement?
                        estBloquante = True
                        minMoves = self.pos[idVoiture] + rh.length[idVoiture] + dist - self.pos[i] #Nombre de coups minimum
                if(self.pos[i] < self.pos[idVoiture] and not right_down): #La voiture est-elle du côté bloquant?
                    if(self.pos[i] + rh.length[i] > self.pos[idVoiture] - dist): #Va-t-elle gêner le déplacement?
                        estBloquante = True
                        minMoves = self.pos[i] + rh.length[i] - (self.pos[idVoiture] - dist) #Nombre de coups minimum
                        
                if(estBloquante):
                    bloquantes.append(i) #On ajoute la voiture bloquante à la liste
                    directionsVoulues.append(right_down) #Cette voiture bloquante devra être déplacée dans le même sens que la voiture courante
                    minMovesNeeded.append(minMoves) 
                    
                    
            if rh.horiz[i] != estHoriz: #Cas d'une voiture bloquante perpendiculaire
                #Y a-t-il intersection entre le déplacement souhaité et la position actuelle de la voiture?
                if (self.pos[i] <= rh.move_on[idVoiture]) and (self.pos[i] + rh.length[i]> rh.move_on[idVoiture]):
                    if(rh.move_on[i] >= self.pos[idVoiture] - dist and not right_down):
                        if(rh.move_on[i] < self.pos[idVoiture]):
                            estBloquante = True
                    elif(rh.move_on[i] < self.pos[idVoiture] + rh.length[idVoiture] + dist and right_down):
                        if(rh.move_on[i] >= self.pos[idVoiture] + rh.length[idVoiture]):
                            estBloquante = True

                    if(estBloquante):

                        bloquantes.append(i) #On ajoute la voiture bloquante à la liste

                        #Cherchons maintenant la direction dans laquelle cette voiture bloquante doit être déplacée
                        direction = False
                        if(rh.length[i] == 2): #Si la voiture bloquante est de longueur 2
                            if(self.pos[i] == 0): #Si la voiture bloquante est collée au bord gauche ou haut
                                direction = True
                            elif(self.pos[i] == rh.move_on[idVoiture] and self.pos[i] < 4): #S'il est plus rapide de l'évacuer par la droite ou le bas
                                direction = True

                        else: #Si la voiture bloquante est de longueur 3 (alors seul un côté du tableau est envisageable)
                            direction = rh.move_on[idVoiture] < 3

                        directionsVoulues.append(direction)

                        #Cherchons enfin un minorant du nombre de déplacements nécessaires pour évacuer cette voiture bloquante
                        if(direction):
                            minMoves = abs(1 + rh.move_on[idVoiture] - self.pos[i])
                        else:
                            minMoves = abs(rh.move_on[idVoiture] - self.pos[i] - rh.length[i])

                        minMovesNeeded.append(minMoves)
                    
        return bloquantes, directionsVoulues, minMovesNeeded
    
    
    """
    Estime le niveau de blocage entre les voitures :
     - calcule un minorant du nombre de coups nécessaires pour évacuer successivement toutes les voitures bloquant
    la voiture d'indice idVoiture, dont on connaît la direction et l'amplitude du déplacement souhaité
     - pénalise la présence de cycles de blocage
    """
    def blocage(self, idVoiture, right_down, dist, cycle, rh):
        count = 0
        bloquantes, directions, movesNeeded = self.chercheBloquantes(idVoiture, right_down, dist, rh)
        n = len(bloquantes)
        firstCar = True
        
        for i in range(n): #Pour chaque voiture bloquante
            if(bloquantes[i] not in self.visitedCars): #On utilise un ensemble pour ne pas considérer plusieurs fois la même voiture dans la chaîne de blocages
                self.visitedCars.add(bloquantes[i])
                
                #On ajoute le nombre de coups nécessaires au minimum pour évacuer cette voiture bloquante
                if(firstCar): #Si c'est la première voiture bloquante trouvée, on continue sur le même cycle
                    cycle.add(bloquantes[i])
                    count += self.blocage(bloquantes[i], directions[i], movesNeeded[i], cycle, rh) + movesNeeded[i] #On cherche récursivement les voitures qui bloquent cette voiture bloquante
                else: #Sinon on entame un nouveau cycle
                    newCycle = set()
                    for car in cycle:
                        newCycle.add(car)
                    newCycle.add(bloquantes[i])
                    count += self.blocage(bloquantes[i], directions[i], movesNeeded[i], newCycle, rh) + movesNeeded[i] #On cherche récursivement les voitures qui bloquent cette voiture bloquante

                firstCar = False
                
            elif(bloquantes[i] in cycle): #On pénalise la présence de cycles
                count += 5
                
        return count
    
            
    """
    Calcul du score de l'état
    
    """
        
    def score_state(self, rh):
        self.score = 0
        self.visitedCars = set()        
        
        #Prise en compte de la roche dans le calcul du score (l'adversaire cherche à maximiser le blocage)
        if self.rock:
            self.score += 2 * self.rock[1] * (5 - abs(self.rock[0] - 2)) #On cherche à placer les roches le plus à droite et le plus proche de la ligne 2 possible

        
        #Calcul du nombre de coups minimal nécessaires pour libérer la voiture rouge (l'algorithme veut minimiser le blocage)
        
        self.visitedCars = set()
        (self.visitedCars).add(0) #On part de la voiture rouge
        dist = 4 - self.pos[0] #Distance restant à parcourir pour la voiture rouge
        
        cycle = set()
        cycle.add(0)
        self.score += dist + self.blocage(0, True, dist, cycle, rh) #La voiture rouge doit se déplacer vers la droite
    
    
    
    """
    Constructeur d'un état à partir du mouvement (c,d)
    """
    
    def move(self, c, d, rh):
        # TODO
        s = State(self.pos)
        s.prev = self
        s.pos[c] += d
        s.c = c
        s.d = d
        s.nb_moves = self.nb_moves + 1
        s.score_state(rh)
        s.rock = self.rock
        s.visitedCars = set()
        return s
    
    
    """
    Constructeur d'un état à partir du positionnement d'une roche
    """

    def put_rock(self, rock_pos):
        s = State(self.pos)
        s.prev = self
        s.pos = self.pos
        s.c = self.c
        s.d = self.d
        s.nb_moves = self.nb_moves
        s.score = self.score
        
        #On actualise la position de la roche
        s.rock = rock_pos
        
        s.visitedCars = set()
        
        return s

    def success(self):
        return self.pos[0] == 4
    
    def __eq__(self, other):
        if not isinstance(other, State):
            return NotImplemented
        if len(self.pos) != len(other.pos):
            print("les états n'ont pas le même nombre de voitures")
        
        return np.array_equal(self.pos, other.pos)
    
    def __hash__(self):
        h = 0
        for i in range(len(self.pos)):
            h = 37*h + self.pos[i]
        return int(h)
    
    def __lt__(self, other):
        return (self.score) < (other.score)


La représentation du problème est faite par la classe **Rushhour** qui contient les champs suivants :

- `nbcars` : le nombre de voitures dans la grille;
- `horiz` : un vecteur contenant un bool indiquant si la voiture bouge horizontalement ou pas;
- `length`: un vecteur contenant la longueur de chaque voiture (2 ou 3);
- `move_on` : un vecteur contenant le numéro de la ligne ou la colonne où se trouve la voiture (0-5);
- `color` : un vecteur contenant le string indiquant la couleur de chaque voiture;
- `free_pos` : une matrice 6x6 contenant un bool permettant de savoir si la case est libre

Toutes les informations pour une voiture se trouvent dans le même index dans chacun des vecteurs.

La fonction *`print_pretty_grid()`* sert à imprimer la grille en utilisant la première lettre des couleurs pour vous aider à mieux visualiser le problème.

In [44]:
class Rushhour:
    
    def __init__(self, horiz, length, move_on, color=None):
        self.nbcars = len(horiz)
        self.horiz = horiz
        self.length = length
        self.move_on = move_on
        self.color = color
        
        self.free_pos = None
    
    def init_positions(self, state):
        self.free_pos = np.ones((6, 6), dtype=bool)
        for i in range(self.nbcars):
            if self.horiz[i]:
                self.free_pos[self.move_on[i], state.pos[i]:state.pos[i]+self.length[i]] = False
            else:
                self.free_pos[state.pos[i]:state.pos[i]+self.length[i], self.move_on[i]] = False
        # TODO
        if state.rock: #Ajout de la roche dans la grille
            self.free_pos[state.rock[0], state.rock[1]] = False
    
    def possible_moves(self, state):
        self.init_positions(state)
        new_states = []
        for i in range(self.nbcars):
            if self.horiz[i]:
                if state.pos[i]+self.length[i]-1 < 5 and self.free_pos[self.move_on[i], state.pos[i]+self.length[i]]:
                    new_states.append(state.move(i, +1, self))
                if state.pos[i] > 0 and self.free_pos[self.move_on[i], state.pos[i] - 1]:
                    new_states.append(state.move(i, -1, self))
            else:
                if state.pos[i]+self.length[i]-1 < 5 and self.free_pos[state.pos[i]+self.length[i], self.move_on[i]]:
                    new_states.append(state.move(i, +1, self))
                if state.pos[i] > 0 and self.free_pos[state.pos[i] - 1, self.move_on[i]]:
                    new_states.append(state.move(i, -1, self))
        return new_states
    
    def possible_rock_moves(self, state):
        self.init_positions(state)
        new_states =[]
        #TODO
        if state.rock: #Si une roche était déjà posée
            for i in range(6):
                if(i!=2 and i!=state.rock[0]): #Interdiction de poser la nouvelle roche sur la ligne 2 ou sur la même ligne que la précédente
                    for j in range(6):
                        if(j!=state.rock[1] and self.free_pos[i, j]): #Interdiction de poser la nouvelle roche sur la même colonne que la précédente ou dans une case occupée
                            new_states.append(state.put_rock((i,j)))
        else:
            for i in range(6):
                if(i!=2):
                    for j in range(6):
                        if(self.free_pos[i, j]):
                                new_states.append(state.put_rock((i,j)))
            
        return new_states
    

    def print_pretty_grid(self, state):
        self.init_positions(state)
        grid= np.chararray((6, 6))
        grid[:]='-'
        for car in range(self.nbcars):
            for pos in range(state.pos[car], state.pos[car] +self.length[car]):
                if self.horiz[car]:
                    grid[self.move_on[car]][pos] = self.color[car][0]
                else:
                    grid[pos][self.move_on[car]] = self.color[car][0]
        if state.rock:
            grid[state.rock] = 'x'
        print(grid)

### 1.1 Implémentation

1. Modifier la classe State pour qu'elle contienne la position de la roche ainsi que la fonction *`put_rock()`* pour ajouter une nouvelle roche et enlever l'ancienne. Les roches sont représentées par un tuple (ligne,colonne). La fonction retourne un nouvel objet State.

2. Modifier la fonction *`init_position()`* de la classe Rushhour pour qu'elle tienne compte de l'emplacement des roches.

3. Implémenter la fonction *`possible_rock_moves()`* qui permet de trouver tous les coups possibles de l'adversaire (avec la roche) à partir de l'état courant dans la classe Rushhour. L'adversaire ne peut ni mettre de roche sur la ligne 2 ni mettre deux roches consécutives sur la même ligne ou colonne.

Utiliser la fonction *testRocks()* pour vérifier que vous avez bien pris en compte les roches. Vous devriez avoir ces résultats:

```
État initial
[[b'-' b'-' b'v' b'j' b'j' b'j']
 [b'o' b'-' b'v' b'v' b'b' b'b']
 [b'o' b'r' b'r' b'v' b'-' b'-']
 [b'-' b'r' b'-' b'v' b'v' b'v']
 [b'v' b'r' b'n' b'n' b'-' b'b']
 [b'v' b'b' b'b' b'b' b'-' b'b']]
[[ True  True False False False False]
 [False  True False False False False]
 [False False False False  True  True]
 [ True False  True False False False]
 [False False False False  True False]
 [False False False False  True False]]


Roche 4-4
[[b'-' b'-' b'v' b'j' b'j' b'j']
 [b'o' b'-' b'v' b'v' b'b' b'b']
 [b'o' b'r' b'r' b'v' b'-' b'-']
 [b'-' b'r' b'-' b'v' b'v' b'v']
 [b'v' b'r' b'n' b'n' b'x' b'b']
 [b'v' b'b' b'b' b'b' b'-' b'b']]
[[ True  True False False False False]
 [False  True False False False False]
 [False False False False  True  True]
 [ True False  True False False False]
 [False False False False False False]
 [False False False False  True False]]


Roche 3-2
[[b'-' b'-' b'v' b'j' b'j' b'j']
 [b'o' b'-' b'v' b'v' b'b' b'b']
 [b'o' b'r' b'r' b'v' b'-' b'-']
 [b'-' b'r' b'x' b'v' b'v' b'v']
 [b'v' b'r' b'n' b'n' b'-' b'b']
 [b'v' b'b' b'b' b'b' b'-' b'b']]
[[ True  True False False False False]
 [False  True False False False False]
 [False False False False  True  True]
 [ True False False False False False]
 [False False False False False False]
 [False False False False  True False]]

```

Utiliser la fonction *testPossibleRockMoves()* pour vérifier que vous trouvez bien tous les coups possibles pour l'adversaire. Vous devriez trouver 7 et 3 mouvements possibles.

In [45]:
def testRocks():
    rh = Rushhour([True, False, True, False, False, True, False, True, False, True, False, True],
                 [2, 2, 3, 2, 3, 2, 2, 2, 2, 2, 2, 3],
                 [2, 2, 0, 0, 3, 1, 1, 3, 0, 4, 5, 5],
                 ["rouge", "vert clair", "jaune", "orange", "violet clair", "bleu ciel", "rose", "violet", "vert", "noir", "beige", "bleu"])
    s0 = State([1, 0, 3, 1, 1, 4, 3, 4, 4, 2, 4, 1])
    
    s1= s0.put_rock((4,4))
    s2 = s1.put_rock((3,2)) 
    
    print("État initial")
    rh.print_pretty_grid(s0)
    print(rh.free_pos)
    print('\n')
    
    print("Roche 4-4")
    rh.print_pretty_grid(s1)
    print(rh.free_pos)
    print('\n')
    
    print("Roche 3-2")
    rh.print_pretty_grid(s2)
    print(rh.free_pos)
    print('\n')

testRocks()

État initial
[[b'-' b'-' b'v' b'j' b'j' b'j']
 [b'o' b'-' b'v' b'v' b'b' b'b']
 [b'o' b'r' b'r' b'v' b'-' b'-']
 [b'-' b'r' b'-' b'v' b'v' b'v']
 [b'v' b'r' b'n' b'n' b'-' b'b']
 [b'v' b'b' b'b' b'b' b'-' b'b']]
[[ True  True False False False False]
 [False  True False False False False]
 [False False False False  True  True]
 [ True False  True False False False]
 [False False False False  True False]
 [False False False False  True False]]


Roche 4-4
[[b'-' b'-' b'v' b'j' b'j' b'j']
 [b'o' b'-' b'v' b'v' b'b' b'b']
 [b'o' b'r' b'r' b'v' b'-' b'-']
 [b'-' b'r' b'-' b'v' b'v' b'v']
 [b'v' b'r' b'n' b'n' b'x' b'b']
 [b'v' b'b' b'b' b'b' b'-' b'b']]
[[ True  True False False False False]
 [False  True False False False False]
 [False False False False  True  True]
 [ True False  True False False False]
 [False False False False False False]
 [False False False False  True False]]


Roche 3-2
[[b'-' b'-' b'v' b'j' b'j' b'j']
 [b'o' b'-' b'v' b'v' b'b' b'b']
 [b'o' b'r' b'r' b'v' b'-' b'

In [46]:
def testPossibleRockMoves():
    rh = Rushhour([True, False, True, False, False, True, False, True, False, True, False, True],
                 [2, 2, 3, 2, 3, 2, 2, 2, 2, 2, 2, 3],
                 [2, 2, 0, 0, 3, 1, 1, 3, 0, 4, 5, 5],
                 ["rouge", "vert clair", "jaune", "orange", "violet clair", "bleu ciel", "rose", "violet", "vert", "noir", "beige", "bleu"])
    s = State([1, 0, 3, 1, 1, 4, 3, 4, 4, 2, 4, 1])
    sols = rh.possible_rock_moves(s)
    print(len(sols))
    s1 = s.put_rock((3,4))
    sols = rh.possible_rock_moves(s1)
    print(len(sols))

testPossibleRockMoves()
  

7
3


## 2. Implémentation d'une recherche minimax (35pts)
L'algorithme minimax est l'implémentation la plus commune pour résoudre un jeu adversarielle avec des mouvements limités définis. Cet algorithme est basé sur le principe de construire un arbre avec les différents états possibles selon les actions de chaque joueur (AI et adversaire). L'algorithme veut maximiser les étages où le AI joue et minimiser les étages où l'adversaire joue. Dans ce cas-ci, l'AI déplace les voitures et l'adversaire place les roches. Dans le cadre du TP, l'algorithme n'explore pas jusqu'aux feuilles de l'arbre. Il s'arrête à une profondeur donnée.

La résolution du problème est faite par la classe **MiniMaxSearch** qui contient les champs suivants:

- `rushhour`: la classe Rushhour qui contient les informations sur la grille;
- `state`: la classe State avec l'état actuel de la grille;
- `search_depth`: la profondeur maximale d'exploration de l'algorithme;


In [66]:
class MiniMaxSearch:
    def __init__(self, rushHour, initial_state, search_depth):
        self.rushhour = rushHour
        self.state = initial_state
        self.search_depth = search_depth
        
        self.visitedStates = set()         #On garde en mémoire les états déjà traversés au cours du jeu pour éviter de les reproduire
        self.visitedStates.add(initial_state)
        
        self.minimax2_states_visited = 0          #Nombre de noeuds visités durant la dernière recherche par minimax normal
        self.minimax2_visited_history=[]  #Stockage, pour les recherches successives de coup optimal, du nombre de noeuds visités durant le minimax normal
        self.minimax_pruning_states_visited = 0   #Nombre de noeuds visités durant la dernière recherche par minimax pruned 
        self.minimax_pruning_visited_history=[]  #Stockage, pour les recherches successives de coup optimal, du nombre de noeuds visités durant le minimax pruned
        
        self.vision_type = "neutral" #Type de joueur (optimistic, neutral, pessimistic) pour l'expectimax
        self.opponent_type = "random" #Type d'adversaire (random, optimal) pour l'expectimax
        
        self.moveCount = 0
        
        
    def set_vision_type(self, vision):
        self.vision_type = vision
        
    def set_opponent_type(self, opponent):
        self.opponent_type = opponent

        
    #L'objet "best_move" est de la forme : (c, d, score)    
    def minimax_1(self, current_depth, current_state): 
        #TODO
        if(current_depth == self.search_depth or current_state.success()): #Si on a atteint la profondeur maximale autorisée, on calcule le score de l'état courant
            current_state.score_state(self.rushhour)
            best_move = (current_state.c, current_state.d, current_state.score)
            
        else: #Sinon on cherche le meilleur mouvement parmi les états accessibles
            best_score = float('Inf')
            bestPossibilities = []
            alreadyVisited = []
                
            self.rushhour.init_positions(current_state)
            stateList = self.rushhour.possible_moves(current_state) 
            for i in range(len(stateList)):
                if stateList[i] not in self.visitedStates:
                    move = self.minimax_1(current_depth + 1, stateList[i])
                    score = move[2]

                    if(score < best_score): #On cherche à minimiser le blocage
                        best_score = score
                        bestPossibilities = [i]
                    elif(score == best_score): #On conserve les états à égalité pour tirer au hasard
                        bestPossibilities.append(i)
                else:
                    alreadyVisited.append(i)
                    
            if bestPossibilities: #On tire au hasard un mouvement parmi les meilleures possibilités
                best_id = random.choice(bestPossibilities)
            else :
                best_id = random.choice(alreadyVisited) #Si aucun nouvel état n'a été trouvé, on se contente de revenir à un état déjà visité
            best_move = (stateList[best_id].c, stateList[best_id].d, best_score)
                
        return best_move
    
    
    #L'objet "best_move" est de la forme : (c, d, score, rock)
    def minimax_2(self, current_depth, current_state, is_max): 
        #TODO
        self.minimax2_states_visited += 1
        
        if(current_depth == self.search_depth or current_state.success()): #Si on a atteint la profondeur maximale autorisée, on calcule le score de l'état courant
            current_state.score_state(self.rushhour)
            best_move = (current_state.c, current_state.d, current_state.score, current_state.rock)
            
        elif is_max : #Si le c'est le tour du joueur (déplacement de voitures)
            best_score = float('Inf')
            bestPossibilities = []
            alreadyVisited = []
                
            self.rushhour.init_positions(current_state)
            stateList = self.rushhour.possible_moves(current_state) 
            for i in range(len(stateList)):
                if stateList[i] not in self.visitedStates:
                    move = self.minimax_2(current_depth + 1, stateList[i], False)
                    if(move[2] < best_score): #On cherche le mouvement qui minimise le blocage
                        best_score = move[2]
                        bestPossibilities = [i]
                    elif(move[2] == best_score): #On conserve les états à égalité pour tirer au hasard
                        bestPossibilities.append(i)
                else:
                    alreadyVisited.append(i)
                    
            if bestPossibilities:
                best_id = random.choice(bestPossibilities)
            else :
                best_id = random.choice(alreadyVisited)
                
            best_move = (stateList[best_id].c, stateList[best_id].d, best_score, stateList[best_id].rock)
        
        else: #Si c'est le tour de l'adversaire (pose de roches)
            worst_score = float('-Inf')
            worstPossibilities = []
            alreadyVisited = []
                
            self.rushhour.init_positions(current_state)
            stateList = self.rushhour.possible_rock_moves(current_state) 
            for i in range(len(stateList)):
                if stateList[i] not in self.visitedStates:
                    move = self.minimax_2(current_depth + 1, stateList[i], True)
                    if(move[2] > worst_score): #On cherche le mouvement qui maximise le blocage
                        worst_score = move[2]
                        worstPossibilities = [i]
                    elif(move[2] == worst_score): #On conserve les états à égalité pour tirer au hasard
                        worstPossibilities.append(i)
                else:
                    alreadyVisited.append(i)
                    
            if worstPossibilities:
                worst_id = random.choice(worstPossibilities)
            else :
                worst_id = random.choice(alreadyVisited)
                
            best_move = (stateList[worst_id].c, stateList[worst_id].d, worst_score, stateList[worst_id].rock)
            
            
        return best_move

    def minimax_pruning(self, current_depth, current_state, is_max, alpha, beta):
        #TODO
        self.minimax_pruning_states_visited += 1
        
        if(current_depth == self.search_depth or current_state.success()): #Si on a atteint la profondeur maximale autorisée, on calcule le score de l'état courant
            current_state.score_state(self.rushhour)
            best_move = (current_state.c, current_state.d, current_state.score, current_state.rock)
        
        elif is_max: #Si c'est un noeud où le joueur joue
            
            best_score = float('Inf')
            best_possible_moves = []
            alreadyVisited = []

            for child in self.rushhour.possible_moves(current_state):
                if child not in self.visitedStates:
                    move = self.minimax_pruning(current_depth + 1, child, False, alpha, beta)

                    if best_score == move[2] :
                        best_possible_moves.append([child.c, child.d, move[2], child.rock])

                    elif best_score > move[2] : #On cherche à minimiser le blocage
                        best_score = move[2]
                        best_possible_moves = [[child.c, child.d, move[2], child.rock]]

                    if alpha >= best_score: #Si possible, on élague la branche
                        if best_possible_moves:
                            best_move = random.choice(best_possible_moves)
                            return best_move

                    beta = min(beta, best_score)
                
                else :
                    alreadyVisited.append(child)
            
            if best_possible_moves:
                best_move = random.choice(best_possible_moves)
            else :
                best_state = random.choice(alreadyVisited)
                best_move = (best_state.c, best_state.d, best_state.score, best_state.rock)
    
    
        else: #Si c'est un noeud où l'adversaire joue
            
            worst_score = float('-Inf')
            worst_possible_moves = []
            alreadyVisited = []

            for child in self.rushhour.possible_rock_moves(current_state):
                if child not in self.visitedStates:
                    move = self.minimax_pruning(current_depth + 1, child, True, alpha, beta)

                    if worst_score == move[2] :
                        worst_possible_moves.append([child.c, child.d, move[2], child.rock])

                    elif worst_score < move[2] : #On cherche à maximiser le blocage
                        worst_score = move[2]
                        worst_possible_moves = [[child.c, child.d, move[2], child.rock]]

                    if beta <= worst_score: #Si possible, on élague la branche
                        if worst_possible_moves:
                            best_move = random.choice(worst_possible_moves)
                            return best_move

                    alpha = max(alpha, worst_score)
                
                else:
                    alreadyVisited.append(child)
                    
            if worst_possible_moves:
                best_move = random.choice(worst_possible_moves)
            else :
                best_state = random.choice(alreadyVisited)
                best_move = (best_state.c, best_state.d, best_state.score, best_state.rock)
                
        return best_move
    

    #L'objet "best_move" est de la forme : (c, d, score, rock, score_max, score_expected, score_min)
    def expectimax(self, current_depth, current_state, is_max):
        #TODO
        
        if(current_depth == self.search_depth or current_state.success()): #Si on a atteint la profondeur maximale autorisée, on calcule le score de l'état courant
            current_state.score_state(self.rushhour)
            best_move = (current_state.c, current_state.d, current_state.score, current_state.rock, current_state.score, current_state.score, current_state.score)
        
        elif is_max: #Choix de l'action du joueur
            
            best_score = float('Inf')
            best_possible_moves = []
            alreadyVisited = []

            for child in self.rushhour.possible_moves(current_state):
                if child not in self.visitedStates:
                    move = self.expectimax(current_depth + 1, child, False)
                    
                    if self.vision_type == "optimistic": # Cas d'un joueur optimiste : on cherche à minimiser le blocage minimal (meilleur cas)
                        current_score = move[6]
                    elif self.vision_type == "neutral": # Cas d'un joueur neutre : on cherche à minimiser l'espérance de blocage
                        current_score = move[5]
                    else: # Cas d'un joueur pessimiste : on cherche à minimiser le blocage maximal (pire cas)
                        current_score = move[4]
                        
                    if best_score == current_score :
                        best_possible_moves.append([child.c, child.d, move[2], child.rock, move[4], move[5], move[6]])

                    elif best_score > current_score : 
                        best_move = [child.c, child.d, move[2], child.rock, move[4], move[5], move[6]]
                        best_possible_moves = [best_move]
                        
                else:
                    alreadyVisited.append(child)

            if best_possible_moves:
                best_move = random.choice(best_possible_moves)
            else :
                best_state = random.choice(alreadyVisited)
                best_move = (best_state.c, best_state.d, best_state.score + 5, best_state.rock, best_state.score + 5, best_state.score + 5, best_state.score + 5)
    
    
        else: #Choix de l'action de l'adversaire (pose de roche)
            
            random_posible_moves = []
            score_max = float('-Inf')
            best_possibility = []
            expected_score = 0
            score_min = float('Inf')
            n = len(self.rushhour.possible_rock_moves(current_state))
            
            for child in self.rushhour.possible_rock_moves(current_state):
                move = self.expectimax(current_depth + 1, child, True)
                random_posible_moves.append(move)
                current_score = move[2]
                
                if current_score > score_max:
                    score_max = current_score
                    best_possibility = [move]
                if current_score < score_min:
                    score_min = current_score
                expected_score += current_score
            
            expected_score = expected_score/n
            
            if self.opponent_type == "random": #Cas d'un adversaire aléatoire
                chosen_move = random.choice(random_posible_moves)
                best_move = (chosen_move[0], chosen_move[1], chosen_move[2], chosen_move[3], score_max, expected_score, score_min)
            
            elif self.opponent_type == "optimal": #Cas d'un adversaire optimal
                if best_possibility:
                    chosen_move = best_possibility[0]
                else:
                    chosen_move = random.choice(random_posible_moves)
                best_move = (chosen_move[0], chosen_move[1], chosen_move[2], chosen_move[3], score_max, expected_score, score_min)

        return best_move

    
    def decide_best_move_1(self):
        #TODO
        best_move = self.minimax_1(0, self.state) #On commence la recherche à une profondeur nulle
        newState = self.state.move(best_move[0], best_move[1], self.rushhour) #On joue le coup optimal
        self.visitedStates.add(newState) #On garde en mémoire l'état obtenu
        return newState

    def decide_best_move_2(self, is_max):
        #TODO
        self.minimax2_visited_history.append(self.minimax2_states_visited)
        
        best_move = self.minimax_2(0, self.state, is_max)
        
        if is_max:
            newState = self.state.move(best_move[0], best_move[1], self.rushhour)
        else : 
            newState = self.state.put_rock(best_move[3])
            
        self.visitedStates.add(newState)
            
        return newState

    def decide_best_move_pruning(self, is_max):
        # TODO
        self.minimax_pruning_visited_history.append(self.minimax_pruning_states_visited)
        
        best_move = self.minimax_pruning(0, self.state, is_max, float('-Inf'), float('Inf'))
        
        if is_max:       
            newState = self.state.move(best_move[0], best_move[1], self.rushhour)
        
        else : 
            newState = self.state.put_rock(best_move[3])
            
        self.visitedStates.add(newState)
            
        return newState

    
    def decide_best_move_expectimax(self, is_max):
        # TODO
        best_move = self.expectimax(0, self.state, is_max)
        
        if is_max:
            newState = self.state.move(best_move[0], best_move[1], self.rushhour)
        
        else : 
            return self.state.put_rock(best_move[3])
        
        self.visitedStates.add(newState)
            
        return newState

    
    def solve(self, state, is_singleplayer, minimax = 'regular'):
        #TODO
        self.moveCount = 0 #Compteur du nombre de coups joués depuis le début de la partie
        
        self.state = state
        self.state.score_state(self.rushhour)
        print(self.state.score)
        self.rushhour.print_pretty_grid(self.state)
        
        if(is_singleplayer):
            while(not self.state.success()):
                newState = self.decide_best_move_1() #On trouve le meilleur coup à jouer

                self.rushhour.init_positions(newState) #On actualise l'objet "rushhour"

                newState.score_state(self.rushhour) #On actualise le score
                print(self.moveCount, " - Score : ", newState.score)

                self.state = newState #On met à jour l'état courant

                self.rushhour.print_pretty_grid(self.state) #On affiche l'état du jeu
                self.print_move(True, self.state)

                self.moveCount +=1
        
        else: #Cas adversariel
            is_max = True
            
            while not self.state.success():
                
                if is_max: 
                    self.moveCount +=1 #Si c'est le tour du joueur, on incrémente le compteur de coups joués
                   
                #Selon l'algorithme souhaité, on détermine le coup à jouer
                if minimax == 'pruning' : 
                    self.state = self.decide_best_move_pruning(is_max)

                elif minimax == 'regular': 
                    self.state = self.decide_best_move_2(is_max)

                elif minimax == 'expectimax':
                    self.state = self.decide_best_move_expectimax(is_max)
                    
                
                self.print_move(is_max,self.state)  #On affiche l'état du jeu
                self.rushhour.print_pretty_grid(self.state)
                is_max = not(is_max) #On change de phase de jeu
            
        print(self.moveCount, " mouvements")

    def print_move(self, is_max, state):
        #TODO
        if(is_max):
            if self.rushhour.horiz[state.c]:
                if state.d == 1:
                    direction = "la droite"
                else:
                    direction = "la gauche"
            else:
                if state.d == 1:
                    direction = "le bas"
                else:
                    direction = "le haut"
    
            print("Voiture", self.rushhour.color[state.c], "vers", direction)
        else:
            print("Rocher en ", state.rock)
          


### 2.1 Implémentation simple
1. Implémenter la fonction *`score_state()`* de la classe **State**. Elle affecte la valeur de l'état à son paramètre score. L'état n'est pas nécessairement final. Utiliser l'heuristique qui vous semble la plus pertinente.

2. Implémenter la fonction  *`minimax_1()`*. Cette fonction contient la logique de l'algorithme minimax pour un seul joueur et retourne le meilleur coup à prendre à partir de l'état courant.

3. Implémenter la fonction *`decide_best_move_1()`*. Cette fonction trouve et exécute le meilleur coup pour une partie à un joueur.

4. Implémenter la fonction *`solve()`*. Cette fonction résout un problème de Rush Hour avec le nombre minimal de coups.

4. Implémenter la fonction *`print_move()`* pour imprimer le coup fait. Ex. "Voiture orange vers le haut" ou "Roche dans la case 2-1".

**N.B**: Vous pouvez modifier les déclarations des fonctions ainsi qu'ajouter des fonctions tant que l'algorithme fonctionne.

Utiliser la fonction _test_print_move()_ pour vérifier votre implémentation de _print_move()_. 

In [50]:
def test_print_move():
    rh = Rushhour([True], [2], [2], ["rouge"])
    s = State([0])
    s = s.put_rock((3,1)) # Roche dans la case 3-1
    s = s.move(0, 1, rh) # Voiture rouge vers la droite

    algo = MiniMaxSearch(rh, s, 1)
    algo.print_move(True, s)
    algo.print_move(False, s)

test_print_move()

Voiture rouge vers la droite
Rocher en  (3, 1)


Pour vous aider dans l'implémentation de votre heuristique, 3 exemples de problèmes de Rush Hour sont fournis.

In [67]:
# Solution optimale: 9 moves
rh = Rushhour([True, False, False, False, True],
                 [2, 3, 2, 3, 3],
                 [2, 4, 5, 1, 5],
                 ["rouge", "vert", "bleu", "orange", "jaune"])
s = State([1, 0, 1, 3, 2])
algo = MiniMaxSearch(rh, s, 1) 
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, True)
%time

[[ True  True  True  True False  True]
 [ True  True  True  True False False]
 [ True False False  True False False]
 [ True False  True  True  True  True]
 [ True False  True  True  True  True]
 [ True False False False False  True]]
14
[[b'-' b'-' b'-' b'-' b'v' b'-']
 [b'-' b'-' b'-' b'-' b'v' b'b']
 [b'-' b'r' b'r' b'-' b'v' b'b']
 [b'-' b'o' b'-' b'-' b'-' b'-']
 [b'-' b'o' b'-' b'-' b'-' b'-']
 [b'-' b'o' b'j' b'j' b'j' b'-']]
0  - Score :  8
[[b'-' b'-' b'-' b'-' b'v' b'-']
 [b'-' b'-' b'-' b'-' b'v' b'b']
 [b'-' b'-' b'r' b'r' b'v' b'b']
 [b'-' b'o' b'-' b'-' b'-' b'-']
 [b'-' b'o' b'-' b'-' b'-' b'-']
 [b'-' b'o' b'j' b'j' b'j' b'-']]
Voiture rouge vers la droite
1  - Score :  7
[[b'-' b'-' b'-' b'-' b'-' b'-']
 [b'-' b'-' b'-' b'-' b'v' b'b']
 [b'-' b'-' b'r' b'r' b'v' b'b']
 [b'-' b'o' b'-' b'-' b'v' b'-']
 [b'-' b'o' b'-' b'-' b'-' b'-']
 [b'-' b'o' b'j' b'j' b'j' b'-']]
Voiture vert vers le bas
2  - Score :  6
[[b'-' b'-' b'-' b'-' b'-' b'b']
 [b'-' b'-' b'-' b'-' b'v' b'b

In [52]:
# solution optimale: 16 moves
rh = Rushhour([True, True, False, False, True, True, False, False],
                 [2, 2, 3, 2, 3, 2, 3, 3],
                 [2, 0, 0, 0, 5, 4, 5, 3],
                 ["rouge", "vert", "mauve", "orange", "emeraude", "lime", "jaune", "bleu"])
s = State([1, 0, 1, 4, 2, 4, 0, 1])
algo = MiniMaxSearch(rh, s, 1) 
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, True) 
%time

[[False False  True  True  True False]
 [False  True  True False  True False]
 [False False False False  True False]
 [False  True  True False  True  True]
 [False  True  True  True False False]
 [False  True False False False  True]]
14
[[b'v' b'v' b'-' b'-' b'-' b'j']
 [b'm' b'-' b'-' b'b' b'-' b'j']
 [b'm' b'r' b'r' b'b' b'-' b'j']
 [b'm' b'-' b'-' b'b' b'-' b'-']
 [b'o' b'-' b'-' b'-' b'l' b'l']
 [b'o' b'-' b'e' b'e' b'e' b'-']]
0  - Score :  13
[[b'-' b'v' b'v' b'-' b'-' b'j']
 [b'm' b'-' b'-' b'b' b'-' b'j']
 [b'm' b'r' b'r' b'b' b'-' b'j']
 [b'm' b'-' b'-' b'b' b'-' b'-']
 [b'o' b'-' b'-' b'-' b'l' b'l']
 [b'o' b'-' b'e' b'e' b'e' b'-']]
Voiture vert vers la droite
1  - Score :  12
[[b'-' b'v' b'v' b'-' b'-' b'j']
 [b'm' b'-' b'-' b'b' b'-' b'j']
 [b'm' b'r' b'r' b'b' b'-' b'j']
 [b'm' b'-' b'-' b'b' b'-' b'-']
 [b'o' b'-' b'-' b'-' b'l' b'l']
 [b'o' b'e' b'e' b'e' b'-' b'-']]
Voiture emeraude vers la gauche
2  - Score :  11
[[b'-' b'v' b'v' b'-' b'-' b'-']
 [b'm' b'-' b'-' b'b'

C:\Users\julie\.julia\conda\3\lib\site-packages\ipykernel_launcher.py:219: RuntimeWarning: overflow encountered in long_scalars


In [53]:
# solution optimale: 14 moves
rh = Rushhour([True, False, True, False, False, False, True, True, False, True, True],
                 [2, 2, 3, 2, 2, 3, 3, 2, 2, 2, 2],
                 [2, 0, 0, 3, 4, 5, 3, 5, 2, 5, 4],
                 ["rouge", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10"])
s = State([0, 0, 3, 1, 2, 1, 0, 0, 4, 3, 4])
algo = MiniMaxSearch(rh, s,1)
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, True)
%time

[[False  True  True False False False]
 [False  True  True False  True False]
 [False False  True False False False]
 [False False False  True False False]
 [ True  True False  True False False]
 [False False False False False  True]]
33
[[b'1' b'-' b'-' b'2' b'2' b'2']
 [b'1' b'-' b'-' b'3' b'-' b'5']
 [b'r' b'r' b'-' b'3' b'4' b'5']
 [b'6' b'6' b'6' b'-' b'4' b'5']
 [b'-' b'-' b'8' b'-' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'-']]
0  - Score :  23
[[b'1' b'-' b'-' b'2' b'2' b'2']
 [b'1' b'-' b'-' b'3' b'4' b'5']
 [b'r' b'r' b'-' b'3' b'4' b'5']
 [b'6' b'6' b'6' b'-' b'-' b'5']
 [b'-' b'-' b'8' b'-' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'-']]
Voiture 4 vers le haut
1  - Score :  11
[[b'1' b'-' b'-' b'2' b'2' b'2']
 [b'1' b'-' b'-' b'-' b'4' b'5']
 [b'r' b'r' b'-' b'3' b'4' b'5']
 [b'6' b'6' b'6' b'3' b'-' b'5']
 [b'-' b'-' b'8' b'-' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'-']]
Voiture 3 vers le bas
2  - Score :  10
[[b'1' b'-' b'-' b'2' b'2' b'2']
 [b'1' b'-' b'-' b'-' b'4' b'5']
 [b

C:\Users\julie\.julia\conda\3\lib\site-packages\ipykernel_launcher.py:219: RuntimeWarning: overflow encountered in long_scalars


### 2.2 Implémentation adversarielle

1. Implémenter la fonction  *`minimax_2()`*. Cette fonction contient la logique de l'algorithme minimax pour deux joueurs et retourne le meilleur coup à prendre à partir de l'état courant.

2. Implémenter la fonction *`decide_best_move_2()`*. Cette fonction trouve et exécute le meilleur coup pour une partie à deux joueurs.

3. Modifier la fonction *`solve()`* pour qu'elle puisse résoudre des problèmes à deux joueurs.

4. Modifier la fonction *`score_state()`* de la classe **State** si elle ne tient pas déjà compte des roches.

**N.B**: Vous pouvez modifier les déclarations des fonctions ainsi qu'ajouter des fonctions tant que l'algorithme fonctionne.

 Le nombre donné de coups pour les 3 exemples suivants correspond à une partie sans adversaire. Le nombre de coups de plus dépend de l'implémentation de votre heuristique.

In [54]:
# Solution optimale: 9 moves
rh = Rushhour([True, False, False, False, True],
                 [2, 3, 2, 3, 3],
                 [2, 4, 5, 1, 5],
                 ["rouge", "vert", "bleu", "orange", "jaune"])
s = State([1, 0, 1, 3, 2])
algo = MiniMaxSearch(rh, s,3)
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, False)
meanAdv1 = round(stat.mean(algo.minimax2_visited_history))
%time

[[ True  True  True  True False  True]
 [ True  True  True  True False False]
 [ True False False  True False False]
 [ True False  True  True  True  True]
 [ True False  True  True  True  True]
 [ True False False False False  True]]
14
[[b'-' b'-' b'-' b'-' b'v' b'-']
 [b'-' b'-' b'-' b'-' b'v' b'b']
 [b'-' b'r' b'r' b'-' b'v' b'b']
 [b'-' b'o' b'-' b'-' b'-' b'-']
 [b'-' b'o' b'-' b'-' b'-' b'-']
 [b'-' b'o' b'j' b'j' b'j' b'-']]
Voiture bleu vers le haut
[[b'-' b'-' b'-' b'-' b'v' b'b']
 [b'-' b'-' b'-' b'-' b'v' b'b']
 [b'-' b'r' b'r' b'-' b'v' b'-']
 [b'-' b'o' b'-' b'-' b'-' b'-']
 [b'-' b'o' b'-' b'-' b'-' b'-']
 [b'-' b'o' b'j' b'j' b'j' b'-']]
Rocher en  (4, 4)
[[b'-' b'-' b'-' b'-' b'v' b'b']
 [b'-' b'-' b'-' b'-' b'v' b'b']
 [b'-' b'r' b'r' b'-' b'v' b'-']
 [b'-' b'o' b'-' b'-' b'-' b'-']
 [b'-' b'o' b'-' b'-' b'x' b'-']
 [b'-' b'o' b'j' b'j' b'j' b'-']]
Voiture vert vers le bas
[[b'-' b'-' b'-' b'-' b'-' b'b']
 [b'-' b'-' b'-' b'-' b'v' b'b']
 [b'-' b'r' b'r' b'-' b'v' b'-

In [56]:
# solution optimale: 16 moves
rh = Rushhour([True, True, False, False, True, True, False, False],
                 [2, 2, 3, 2, 3, 2, 3, 3],
                 [2, 0, 0, 0, 5, 4, 5, 3],
                 ["rouge", "vert", "mauve", "orange", "emeraude", "lime", "jaune", "bleu"])
s = State([1, 0, 1, 4, 2, 4, 0, 1])
algo = MiniMaxSearch(rh, s, 3) 
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, False) 
meanAdv2 = round(stat.mean(algo.minimax2_visited_history))
%time

C:\Users\julie\.julia\conda\3\lib\site-packages\ipykernel_launcher.py:219: RuntimeWarning: overflow encountered in long_scalars


[[False False  True  True  True False]
 [False  True  True False  True False]
 [False False False False  True False]
 [False  True  True False  True  True]
 [False  True  True  True False False]
 [False  True False False False  True]]
14
[[b'v' b'v' b'-' b'-' b'-' b'j']
 [b'm' b'-' b'-' b'b' b'-' b'j']
 [b'm' b'r' b'r' b'b' b'-' b'j']
 [b'm' b'-' b'-' b'b' b'-' b'-']
 [b'o' b'-' b'-' b'-' b'l' b'l']
 [b'o' b'-' b'e' b'e' b'e' b'-']]
Voiture jaune vers le bas
[[b'v' b'v' b'-' b'-' b'-' b'-']
 [b'm' b'-' b'-' b'b' b'-' b'j']
 [b'm' b'r' b'r' b'b' b'-' b'j']
 [b'm' b'-' b'-' b'b' b'-' b'j']
 [b'o' b'-' b'-' b'-' b'l' b'l']
 [b'o' b'-' b'e' b'e' b'e' b'-']]
Rocher en  (4, 3)
[[b'v' b'v' b'-' b'-' b'-' b'-']
 [b'm' b'-' b'-' b'b' b'-' b'j']
 [b'm' b'r' b'r' b'b' b'-' b'j']
 [b'm' b'-' b'-' b'b' b'-' b'j']
 [b'o' b'-' b'-' b'x' b'l' b'l']
 [b'o' b'-' b'e' b'e' b'e' b'-']]
Voiture emeraude vers la droite
[[b'v' b'v' b'-' b'-' b'-' b'-']
 [b'm' b'-' b'-' b'b' b'-' b'j']
 [b'm' b'r' b'r' b'b' b

In [57]:
# solution optimale: 14 moves
rh = Rushhour([True, False, True, False, False, False, True, True, False, True, True],
                 [2, 2, 3, 2, 2, 3, 3, 2, 2, 2, 2],
                 [2, 0, 0, 3, 4, 5, 3, 5, 2, 5, 4],
                 ["rouge", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10"])
s = State([0, 0, 3, 1, 2, 1, 0, 0, 4, 3, 4])
algo = MiniMaxSearch(rh, s,3)
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, False)
meanAdv3 = round(stat.mean(algo.minimax2_visited_history))
%time

C:\Users\julie\.julia\conda\3\lib\site-packages\ipykernel_launcher.py:219: RuntimeWarning: overflow encountered in long_scalars


[[False  True  True False False False]
 [False  True  True False  True False]
 [False False  True False False False]
 [False False False  True False False]
 [ True  True False  True False False]
 [False False False False False  True]]
33
[[b'1' b'-' b'-' b'2' b'2' b'2']
 [b'1' b'-' b'-' b'3' b'-' b'5']
 [b'r' b'r' b'-' b'3' b'4' b'5']
 [b'6' b'6' b'6' b'-' b'4' b'5']
 [b'-' b'-' b'8' b'-' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'-']]
Voiture 4 vers le haut
[[b'1' b'-' b'-' b'2' b'2' b'2']
 [b'1' b'-' b'-' b'3' b'4' b'5']
 [b'r' b'r' b'-' b'3' b'4' b'5']
 [b'6' b'6' b'6' b'-' b'-' b'5']
 [b'-' b'-' b'8' b'-' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'-']]
Rocher en  (4, 3)
[[b'1' b'-' b'-' b'2' b'2' b'2']
 [b'1' b'-' b'-' b'3' b'4' b'5']
 [b'r' b'r' b'-' b'3' b'4' b'5']
 [b'6' b'6' b'6' b'-' b'-' b'5']
 [b'-' b'-' b'8' b'x' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'-']]
Voiture rouge vers la droite
[[b'1' b'-' b'-' b'2' b'2' b'2']
 [b'1' b'-' b'-' b'3' b'4' b'5']
 [b'-' b'r' b'r' b'3' b'4' b'

## 3. Élagage Alpha-Beta (15pts)
Pour accélérer la prise de décision, un élagage est effectué sur l'arbre. Cela permet de ne pas parcourir tout l'arbre. L'élagage alpha-beta est l'implémentation la plus fréquente pour résoudre ce genre de problème. En passant des valeurs d’alpha et de bêta récursivement, l'algorithme connaît la meilleure valeur trouvée à date. Les branches jugées non intéressantes, c'est-à-dire qu'elles ne peuvent pas produire un meilleur résultat que celui déjà trouvé, sont coupées dans le processus.

 
### 3.1 Implémentation

1. Implémenter la fonction *`minimax_pruning()`* pour qu'elle n'explore pas des branches inutiles lors d'une partie à deux joueurs.
2. Implémenter la fonction *`decide_best_move_pruning()`*. Cette fonction trouve et exécute le meilleur coup pour une partie à deux joueurs.
3. Modifier la fonction *`solve()`* pour qu'elle utilise cette nouvelle méthode.
4. Vérifier que cette nouvelle méthode visite bel et bien moins de noeuds. **Montrer cette information dans un tableau**.

In [58]:
# solution optimale: 9 moves
rh = Rushhour([True, False, False, False, True],
                 [2, 3, 2, 3, 3],
                 [2, 4, 5, 1, 5],
                 ["rouge", "vert", "bleu", "orange", "jaune"])
s = State([1, 0, 1, 3, 2])
algo = MiniMaxSearch(rh, s, 3)
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, False, 'pruning')
meanPrun1 = round(stat.mean(algo.minimax_pruning_visited_history))
%time

[[ True  True  True  True False  True]
 [ True  True  True  True False False]
 [ True False False  True False False]
 [ True False  True  True  True  True]
 [ True False  True  True  True  True]
 [ True False False False False  True]]
14
[[b'-' b'-' b'-' b'-' b'v' b'-']
 [b'-' b'-' b'-' b'-' b'v' b'b']
 [b'-' b'r' b'r' b'-' b'v' b'b']
 [b'-' b'o' b'-' b'-' b'-' b'-']
 [b'-' b'o' b'-' b'-' b'-' b'-']
 [b'-' b'o' b'j' b'j' b'j' b'-']]
Voiture vert vers le bas
[[b'-' b'-' b'-' b'-' b'-' b'-']
 [b'-' b'-' b'-' b'-' b'v' b'b']
 [b'-' b'r' b'r' b'-' b'v' b'b']
 [b'-' b'o' b'-' b'-' b'v' b'-']
 [b'-' b'o' b'-' b'-' b'-' b'-']
 [b'-' b'o' b'j' b'j' b'j' b'-']]
Rocher en  (4, 3)
[[b'-' b'-' b'-' b'-' b'-' b'-']
 [b'-' b'-' b'-' b'-' b'v' b'b']
 [b'-' b'r' b'r' b'-' b'v' b'b']
 [b'-' b'o' b'-' b'-' b'v' b'-']
 [b'-' b'o' b'-' b'x' b'-' b'-']
 [b'-' b'o' b'j' b'j' b'j' b'-']]
Voiture bleu vers le haut
[[b'-' b'-' b'-' b'-' b'-' b'b']
 [b'-' b'-' b'-' b'-' b'v' b'b']
 [b'-' b'r' b'r' b'-' b'v' b'-

In [60]:
# solution optimale: 16 moves
rh = Rushhour([True, True, False, False, True, True, False, False],
                 [2, 2, 3, 2, 3, 2, 3, 3],
                 [2, 0, 0, 0, 5, 4, 5, 3],
                 ["rouge", "vert", "mauve", "orange", "emeraude", "lime", "jaune", "bleu"])
s = State([1, 0, 1, 4, 2, 4, 0, 1])
algo = MiniMaxSearch(rh, s, 3) 
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, False, 'pruning')
meanPrun2 = round(stat.mean(algo.minimax_pruning_visited_history))
%time

C:\Users\julie\.julia\conda\3\lib\site-packages\ipykernel_launcher.py:219: RuntimeWarning: overflow encountered in long_scalars


[[False False  True  True  True False]
 [False  True  True False  True False]
 [False False False False  True False]
 [False  True  True False  True  True]
 [False  True  True  True False False]
 [False  True False False False  True]]
14
[[b'v' b'v' b'-' b'-' b'-' b'j']
 [b'm' b'-' b'-' b'b' b'-' b'j']
 [b'm' b'r' b'r' b'b' b'-' b'j']
 [b'm' b'-' b'-' b'b' b'-' b'-']
 [b'o' b'-' b'-' b'-' b'l' b'l']
 [b'o' b'-' b'e' b'e' b'e' b'-']]
Voiture jaune vers le bas
[[b'v' b'v' b'-' b'-' b'-' b'-']
 [b'm' b'-' b'-' b'b' b'-' b'j']
 [b'm' b'r' b'r' b'b' b'-' b'j']
 [b'm' b'-' b'-' b'b' b'-' b'j']
 [b'o' b'-' b'-' b'-' b'l' b'l']
 [b'o' b'-' b'e' b'e' b'e' b'-']]
Rocher en  (0, 3)
[[b'v' b'v' b'-' b'x' b'-' b'-']
 [b'm' b'-' b'-' b'b' b'-' b'j']
 [b'm' b'r' b'r' b'b' b'-' b'j']
 [b'm' b'-' b'-' b'b' b'-' b'j']
 [b'o' b'-' b'-' b'-' b'l' b'l']
 [b'o' b'-' b'e' b'e' b'e' b'-']]
Voiture emeraude vers la droite
[[b'v' b'v' b'-' b'x' b'-' b'-']
 [b'm' b'-' b'-' b'b' b'-' b'j']
 [b'm' b'r' b'r' b'b' b

In [61]:
# solution optimale: 14 moves
rh = Rushhour([True, False, True, False, False, False, True, True, False, True, True],
                 [2, 2, 3, 2, 2, 3, 3, 2, 2, 2, 2],
                 [2, 0, 0, 3, 4, 5, 3, 5, 2, 5, 4],
                 ["rouge", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10"])
s = State([0, 0, 3, 1, 2, 1, 0, 0, 4, 3, 4])
algo = MiniMaxSearch(rh, s,3)
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, False, 'pruning')
meanPrun3 = round(stat.mean(algo.minimax_pruning_visited_history))
%time

C:\Users\julie\.julia\conda\3\lib\site-packages\ipykernel_launcher.py:219: RuntimeWarning: overflow encountered in long_scalars


[[False  True  True False False False]
 [False  True  True False  True False]
 [False False  True False False False]
 [False False False  True False False]
 [ True  True False  True False False]
 [False False False False False  True]]
33
[[b'1' b'-' b'-' b'2' b'2' b'2']
 [b'1' b'-' b'-' b'3' b'-' b'5']
 [b'r' b'r' b'-' b'3' b'4' b'5']
 [b'6' b'6' b'6' b'-' b'4' b'5']
 [b'-' b'-' b'8' b'-' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'-']]
Voiture 4 vers le haut
[[b'1' b'-' b'-' b'2' b'2' b'2']
 [b'1' b'-' b'-' b'3' b'4' b'5']
 [b'r' b'r' b'-' b'3' b'4' b'5']
 [b'6' b'6' b'6' b'-' b'-' b'5']
 [b'-' b'-' b'8' b'-' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'-']]
Rocher en  (0, 2)
[[b'1' b'-' b'x' b'2' b'2' b'2']
 [b'1' b'-' b'-' b'3' b'4' b'5']
 [b'r' b'r' b'-' b'3' b'4' b'5']
 [b'6' b'6' b'6' b'-' b'-' b'5']
 [b'-' b'-' b'8' b'-' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'-']]
Voiture 9 vers la droite
[[b'1' b'-' b'x' b'2' b'2' b'2']
 [b'1' b'-' b'-' b'3' b'4' b'5']
 [b'r' b'r' b'-' b'3' b'4' b'5']


In [62]:
#Comparaison du minimax normal et du minimax avec pruning : 
print("Nombre moyen de noeuds traversés pour une recherche de mouvement optimal :")
print("\n")

print("Algo", "\t\t", "Problème 1 (9 moves)", "\t", "Problème 2 (16 moves)", "\t", "Problème 3 (14 moves)")
print(" ")
print("Normal", "\t\t", meanAdv1, "\t\t\t", meanAdv2, "\t\t\t", meanAdv3)
print("Pruning", "\t", meanPrun1, "\t\t\t", meanPrun2, "\t\t\t", meanPrun3)


Nombre moyen de noeuds traversés pour une recherche de mouvement optimal :


Algo 		 Problème 1 (9 moves) 	 Problème 2 (16 moves) 	 Problème 3 (14 moves)
 
Normal 		 2230 			 5727 			 5596
Pruning 	 1147 			 2081 			 2099


## 4. Expectimax (15 pts)
L'expectimax est une variante de la recherche minimax. Elle est utilisée lorsqu'il y a une composante de hasard lors de la prise de décision. Depuis le début du TP, il a été supposé que l'adversaire prenait toujours le coup optimal. Maintenant, l'adversaire va faire un coup aléatoirement. Pour résoudre ce cas-là, la recherche expectimax sera utilisée.

### 4.1 Implémentation 

1. Implémenter la fonction *`minimax_expectimax()`* pour qu'elle implémente l'algorithme de recherche. Considérer une probabilité égale pour tous les coups.
2. Implémenter la fonction *`decide_best_move_expectimax()`*. Cette fonction trouve et exécute le meilleur coup pour une partie avec un adversaire imprévisible.
3. Modifier la fonction *`solve()`* pour qu'elle utilise cette nouvelle méthode.
4. Modifier les valeurs de probabilités associées aux coups pour donner une vision optimiste et une vision pessimiste à l'AI. Comparer la performance du AI entre les 3 visions (aléatoire, optimiste, pessimiste). Attention l'adversaire continue à prendre ses décisions au hasard. **Montrer ces informations dans un tableau et les analyser**. 

In [65]:
# solution optimale: 9 moves
rh = Rushhour([True, False, False, False, True],
                 [2, 3, 2, 3, 3],
                 [2, 4, 5, 1, 5],
                 ["rouge", "vert", "bleu", "orange", "jaune"])
s = State([1, 0, 1, 3, 2])
algo = MiniMaxSearch(rh, s,3)
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, False, 'expectimax')

%time

[[ True  True  True  True False  True]
 [ True  True  True  True False False]
 [ True False False  True False False]
 [ True False  True  True  True  True]
 [ True False  True  True  True  True]
 [ True False False False False  True]]
14
[[b'-' b'-' b'-' b'-' b'v' b'-']
 [b'-' b'-' b'-' b'-' b'v' b'b']
 [b'-' b'r' b'r' b'-' b'v' b'b']
 [b'-' b'o' b'-' b'-' b'-' b'-']
 [b'-' b'o' b'-' b'-' b'-' b'-']
 [b'-' b'o' b'j' b'j' b'j' b'-']]
Voiture jaune vers la droite
[[b'-' b'-' b'-' b'-' b'v' b'-']
 [b'-' b'-' b'-' b'-' b'v' b'b']
 [b'-' b'r' b'r' b'-' b'v' b'b']
 [b'-' b'o' b'-' b'-' b'-' b'-']
 [b'-' b'o' b'-' b'-' b'-' b'-']
 [b'-' b'o' b'-' b'j' b'j' b'j']]
Rocher en  (5, 0)
[[b'-' b'-' b'-' b'-' b'v' b'-']
 [b'-' b'-' b'-' b'-' b'v' b'b']
 [b'-' b'r' b'r' b'-' b'v' b'b']
 [b'-' b'o' b'-' b'-' b'-' b'-']
 [b'-' b'o' b'-' b'-' b'-' b'-']
 [b'x' b'o' b'-' b'j' b'j' b'j']]
Voiture bleu vers le haut
[[b'-' b'-' b'-' b'-' b'v' b'b']
 [b'-' b'-' b'-' b'-' b'v' b'b']
 [b'-' b'r' b'r' b'-' b'v'

In [39]:
# solution optimale: 16 moves
rh = Rushhour([True, True, False, False, True, True, False, False],
                 [2, 2, 3, 2, 3, 2, 3, 3],
                 [2, 0, 0, 0, 5, 4, 5, 3],
                 ["rouge", "vert", "mauve", "orange", "emeraude", "lime", "jaune", "bleu"])
s = State([1, 0, 1, 4, 2, 4, 0, 1])
algo = MiniMaxSearch(rh, s, 3) 
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, False, 'expectimax') 
%time

C:\Users\julie\.julia\conda\3\lib\site-packages\ipykernel_launcher.py:246: RuntimeWarning: overflow encountered in long_scalars


[[False False  True  True  True False]
 [False  True  True False  True False]
 [False False False False  True False]
 [False  True  True False  True  True]
 [False  True  True  True False False]
 [False  True False False False  True]]
14
[[b'v' b'v' b'-' b'-' b'-' b'j']
 [b'm' b'-' b'-' b'b' b'-' b'j']
 [b'm' b'r' b'r' b'b' b'-' b'j']
 [b'm' b'-' b'-' b'b' b'-' b'-']
 [b'o' b'-' b'-' b'-' b'l' b'l']
 [b'o' b'-' b'e' b'e' b'e' b'-']]
Voiture bleu vers le haut
[[b'v' b'v' b'-' b'b' b'-' b'j']
 [b'm' b'-' b'-' b'b' b'-' b'j']
 [b'm' b'r' b'r' b'b' b'-' b'j']
 [b'm' b'-' b'-' b'-' b'-' b'-']
 [b'o' b'-' b'-' b'-' b'l' b'l']
 [b'o' b'-' b'e' b'e' b'e' b'-']]
Rocher en  (1, 4)
[[b'v' b'v' b'-' b'b' b'-' b'j']
 [b'm' b'-' b'-' b'b' b'x' b'j']
 [b'm' b'r' b'r' b'b' b'-' b'j']
 [b'm' b'-' b'-' b'-' b'-' b'-']
 [b'o' b'-' b'-' b'-' b'l' b'l']
 [b'o' b'-' b'e' b'e' b'e' b'-']]
Voiture jaune vers le bas
[[b'v' b'v' b'-' b'b' b'-' b'-']
 [b'm' b'-' b'-' b'b' b'x' b'j']
 [b'm' b'r' b'r' b'b' b'-' b'

In [40]:
# solution optimale: 14 moves
rh = Rushhour([True, False, True, False, False, False, True, True, False, True, True],
                 [2, 2, 3, 2, 2, 3, 3, 2, 2, 2, 2],
                 [2, 0, 0, 3, 4, 5, 3, 5, 2, 5, 4],
                 ["rouge", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10"])
s = State([0, 0, 3, 1, 2, 1, 0, 0, 4, 3, 4])
algo = MiniMaxSearch(rh, s, 3)
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, False, 'expectimax')
%time

C:\Users\julie\.julia\conda\3\lib\site-packages\ipykernel_launcher.py:246: RuntimeWarning: overflow encountered in long_scalars


[[False  True  True False False False]
 [False  True  True False  True False]
 [False False  True False False False]
 [False False False  True False False]
 [ True  True False  True False False]
 [False False False False False  True]]
33
[[b'1' b'-' b'-' b'2' b'2' b'2']
 [b'1' b'-' b'-' b'3' b'-' b'5']
 [b'r' b'r' b'-' b'3' b'4' b'5']
 [b'6' b'6' b'6' b'-' b'4' b'5']
 [b'-' b'-' b'8' b'-' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'-']]
Voiture 10 vers la gauche
[[b'1' b'-' b'-' b'2' b'2' b'2']
 [b'1' b'-' b'-' b'3' b'-' b'5']
 [b'r' b'r' b'-' b'3' b'4' b'5']
 [b'6' b'6' b'6' b'-' b'4' b'5']
 [b'-' b'-' b'8' b'1' b'1' b'-']
 [b'7' b'7' b'8' b'9' b'9' b'-']]
Rocher en  (4, 1)
[[b'1' b'-' b'-' b'2' b'2' b'2']
 [b'1' b'-' b'-' b'3' b'-' b'5']
 [b'r' b'r' b'-' b'3' b'4' b'5']
 [b'6' b'6' b'6' b'-' b'4' b'5']
 [b'-' b'x' b'8' b'1' b'1' b'-']
 [b'7' b'7' b'8' b'9' b'9' b'-']]
Voiture 9 vers la droite
[[b'1' b'-' b'-' b'2' b'2' b'2']
 [b'1' b'-' b'-' b'3' b'-' b'5']
 [b'r' b'r' b'-' b'3' b'4' b'5

KeyboardInterrupt: 

In [70]:
#Comparaison des visions (optimiste / neutre / pessimiste)
rh = Rushhour([True, False, False, False, True],
                 [2, 3, 2, 3, 3],
                 [2, 4, 5, 1, 5],
                 ["rouge", "vert", "bleu", "orange", "jaune"])

s = State([1, 0, 1, 3, 2])
algo = MiniMaxSearch(rh, s,3)

algo.rushhour.init_positions(s)
algo.set_vision_type("optimistic")
algo.solve(s, False, 'expectimax')
opti = algo.moveCount

algo.rushhour.init_positions(s)
algo.set_vision_type("neutral")
algo.solve(s, False, 'expectimax')
neutr = algo.moveCount

algo.rushhour.init_positions(s)
algo.set_vision_type("pessimistic")
algo.solve(s, False, 'expectimax')
pessi = algo.moveCount

print("Vision\t\t Nombre de mouvements")
print("Optimiste\t", opti)
print("Neutre\t\t", neutr)
print("Pessimiste\t", pessi)

%time

14
[[b'-' b'-' b'-' b'-' b'v' b'-']
 [b'-' b'-' b'-' b'-' b'v' b'b']
 [b'-' b'r' b'r' b'-' b'v' b'b']
 [b'-' b'o' b'-' b'-' b'-' b'-']
 [b'-' b'o' b'-' b'-' b'-' b'-']
 [b'-' b'o' b'j' b'j' b'j' b'-']]
Voiture jaune vers la droite
[[b'-' b'-' b'-' b'-' b'v' b'-']
 [b'-' b'-' b'-' b'-' b'v' b'b']
 [b'-' b'r' b'r' b'-' b'v' b'b']
 [b'-' b'o' b'-' b'-' b'-' b'-']
 [b'-' b'o' b'-' b'-' b'-' b'-']
 [b'-' b'o' b'-' b'j' b'j' b'j']]
Rocher en  (4, 2)
[[b'-' b'-' b'-' b'-' b'v' b'-']
 [b'-' b'-' b'-' b'-' b'v' b'b']
 [b'-' b'r' b'r' b'-' b'v' b'b']
 [b'-' b'o' b'-' b'-' b'-' b'-']
 [b'-' b'o' b'x' b'-' b'-' b'-']
 [b'-' b'o' b'-' b'j' b'j' b'j']]
Voiture bleu vers le haut
[[b'-' b'-' b'-' b'-' b'v' b'b']
 [b'-' b'-' b'-' b'-' b'v' b'b']
 [b'-' b'r' b'r' b'-' b'v' b'-']
 [b'-' b'o' b'-' b'-' b'-' b'-']
 [b'-' b'o' b'x' b'-' b'-' b'-']
 [b'-' b'o' b'-' b'j' b'j' b'j']]
Rocher en  (0, 3)
[[b'-' b'-' b'-' b'x' b'v' b'b']
 [b'-' b'-' b'-' b'-' b'v' b'b']
 [b'-' b'r' b'r' b'-' b'v' b'-']
 [b'-' b'o'

## 5. Comparaison (10pts)
Comparez la performance la recherche minimax avec la recherche expectimax lorsque l'adversaire est optimale et lorsque l'adversaire est aléatoire. Pour un adversaire aléatoire, prenez une vision neutre. Roulez plusieurs fois les algorithmes (vous pouvez aussi reprendre les tests du TP1) et notez si c'est une victoire ou une défaire pour l'AI ainsi que le nombre de coups moyen, minimal et maximal. Basez-vous sur le nombre de coups optimaux pour résoudre les problèmes pour déterminer un seuil de victoire et de défaite pour l'AI. Précisez votre seuil. **Mettez le tout dans un tableau et analysez vos résultats**. 